In [139]:
import pandas as pd
import os
import spacy
import re
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Get Data

In [140]:
# Get data
data = pd.DataFrame()
current_dir = os.getcwd()
for dir in ['train', 'test', 'validation']:
    df = pd.read_csv(f"{os.path.join(current_dir, 'data', dir)}.csv")
    print(f'File {dir}.csv có {df.shape[0]} dòng.')
    data = pd.concat([data,df], axis=0)
print(f'Data tổng có {data.shape[0]} dòng.')

File train.csv có 11426 dòng.
File test.csv có 3166 dòng.
File validation.csv có 1583 dòng.
Data tổng có 16175 dòng.


In [141]:
data['sentiment'].value_counts()

,count
sentiment,
2,8038
0,7439
1,698


In [142]:
# Preview data
data.sample(10)

,sentence,sentiment,topic
1826,sự tận tình đối với học viên .,2,0
4779,thầy nhiệt tình trả lời câu hỏi của sinh viên ...,2,0
4638,đôi khi em nghĩ thầy cho deadline quá trời như...,0,0
1692,cần phải có phương pháp giảng dạy linh hoạt và...,0,0
1183,"cô điểm danh mình em lúc đầu giờ , cuối giờ mớ...",0,0
7278,"khó hiểu , khó vận dụng .",0,3
4486,"nên cập nhật giáo trình đầy đủ , nội dung môn ...",0,1
11276,"ít dự án , bài tập liên quan đến thực tế .",0,1
8145,em đề nghị đổi phòng học .,0,2
11063,"dạy tốt , vui tính .",2,0


In [143]:
# Convert sentiment and topic
# Convert sentiment and topic column
sentiment = {'0':'negative', '1':'neutral', '2':'positive'}
topic = {'0':'lecturer', '1':'training_program', '2':'facility', '3':'others'}

data['target'] = data['sentiment'].astype(str).map(sentiment)
data['topic_'] = data['topic'].astype(str).map(topic)

data.sample(10)

,sentence,sentiment,topic,target,topic_
1290,phòng máy cũ cần nâng cấp .,0,2,negative,facility
9918,"thiếu lôi cuốn , còn nói dựa vào slide nhiều .",0,0,negative,lecturer
2177,giảng hơi nhiều kiến thức vượt quá phạm vi môn...,0,0,negative,lecturer
4712,giảng viên giải đáp các thắc mắc của sinh viên...,2,0,positive,lecturer
2879,"thầy rất hiền , nhiệt tình .",2,0,positive,lecturer
540,thêm phần anh văn chuyên ngành vào nội dung học .,0,1,negative,training_program
2947,thầy dạy chưa chủ động giải đáp các thắc mắc l...,0,0,negative,lecturer
10579,tài liệu chính thức của trường chưa có .,0,1,negative,training_program
4182,thầy giảng hơi nhanh .,0,0,negative,lecturer
1104,chỉ gặp được thầy đúng một lần trong suốt học ...,0,0,negative,lecturer


# Preprocess

In [144]:
# Remove sentiment neutral
data = data[data['sentiment'] != 1]
data.shape

(15477, 5)

In [145]:
data['sentiment'].value_counts()

,count
sentiment,
2,8038
0,7439


In [146]:
# Convert to lower
data.loc[:, 'sentence'] = data['sentence'].str.lower()

In [147]:
# Remove punctuation
data.loc[:, 'sentence'] = data['sentence'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

In [148]:
# Tokenization
data['tokens'] = data['sentence'].apply(lambda x: ViTokenizer.tokenize(x))

In [130]:
# Remove stopwords
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [line.strip().lower() for line in f if line.strip()]
stopwords = ['_'.join(w.split()) for w in stopwords]
data['tokens'] = data['tokens'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))

In [131]:
data

,sentence,sentiment,topic,target,topic_,tokens
0,slide giáo trình đầy đủ,2,1,positive,training_program,slide giáo_trình đầy_đủ
1,nhiệt tình giảng dạy gần gũi với sinh viên,2,0,positive,lecturer,nhiệt_tình giảng_dạy gần_gũi sinh_viên
2,đi học đầy đủ full điểm chuyên cần,0,1,negative,training_program,đi học đầy_đủ full chuyên_cần
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0,negative,lecturer,chưa áp_dụng công_nghệ thông_tin thiết_bị giản...
4,thầy giảng bài hay có nhiều bài tập ví dụ nga...,2,0,positive,lecturer,thầy giảng bài_tập ví_dụ lớp
...,...,...,...,...,...,...
1578,hướng dẫn lab mơ hồ,0,0,negative,lecturer,hướng_dẫn lab mơ_hồ
1579,thầy cho chúng em những bài tập mang tính thực...,2,0,positive,lecturer,thầy bài_tập thực_hành thực_tiễn hài_lòng
1580,thầy không dạy nhiều chủ yếu cho sinh viên tự ...,0,0,negative,lecturer,thầy không dạy chủ_yếu sinh_viên
1581,em muốn đổi tên môn học vì tên môn là lập trìn...,0,1,negative,training_program,đổi môn_học môn lập_trình c fraction cplusplus...


# Build Model

In [132]:
X, y = data['tokens'], data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [133]:
# TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_train_vectorizer = vectorizer.fit_transform(X_train)
X_test_vectorizer = vectorizer.transform(X_test)

In [134]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vectorizer, y_train)

LogisticRegression(max_iter=1000)

In [135]:
y_pred = model.predict(X_test_vectorizer)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

cfm = confusion_matrix(y_test, y_pred)
print(cfm)

report = classification_report(y_test, y_pred)
print(report)

Accuracy: 0.8729543496985357
[[1973  258]
 [ 332 2081]]
              precision    recall  f1-score   support

           0       0.86      0.88      0.87      2231
           2       0.89      0.86      0.88      2413

    accuracy                           0.87      4644
   macro avg       0.87      0.87      0.87      4644
weighted avg       0.87      0.87      0.87      4644



# Save Model

In [136]:
import joblib

In [137]:
current_dir = os.getcwd()
save_dir = os.path.join(current_dir, 'model')
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

joblib.dump(model, os.path.join(save_dir, 'model.pkl'))
joblib.dump(vectorizer, os.path.join(save_dir, 'vectorizer.pkl'))

['/content/model/vectorizer.pkl']